## Define Variables

In [1]:
workingdir = r"C:\Users\Renato\Dropbox\Personal\Graduate School\Princeton\Conferences\SAMSI Workshop 2017 - Research Triangle Park NC\Code"
Datapath = workingdir + '\\Data'

## Define Training and Testing Data Sets

In [2]:
import scipy.io as spio
import numpy as np
import random as random
import scipy.stats as stats
import pandas as pd
df=pd.read_csv(Datapath + '\\Data_HotOneEncodedColony.csv', sep=',')
#col_inputs = ['A12_1', 'A12_2', 'M12_1', 'M12_6', 'ML5126', 'HeadToThorax_mm','Corpulence_mm','Dist']
col_inputs = ['HeadToThorax_mm','Corpulence_mm','Dist']
INPUTS = df[col_inputs].values
OUTPUT = df[['Activity']].values.ravel()

Activity = ['b', 'bc', 'f', 'fd', 'go', 'i', 'og', 'sg', 't', 'wi', 'wo'];

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
dfn.hist()
plt.show()


## Decision Tree (75% training, 25% testing)

In [45]:
# group labels to reduce overall number of tasks
dfMod = df.copy()
# Group 0: b (Nest Building)
# dfMod.loc[dfMod['Activity'] == 0, 'Activity'] = 0
# Group 1: bc (Brood Care)
# dfMod.loc[dfMod['Activity'] == 1, 'Activity'] = 1
# Group 2: wo+f (Outside)
dfMod.loc[(dfMod['Activity'] == 10) | (dfMod['Activity'] == 2), 'Activity'] = 2
# Group 3: sg+fd (Selfish)
dfMod.loc[(dfMod['Activity'] == 7) | (dfMod['Activity'] == 3), 'Activity'] = 3
# Group 4: t+go+og (Interaction)
dfMod.loc[(dfMod['Activity'] == 8) | (dfMod['Activity'] == 4) | (dfMod['Activity'] == 6), 'Activity'] = 4
# Group 5: i (Inactive)
dfMod.loc[(dfMod['Activity'] == 5) | (dfMod['Activity'] == 9), 'Activity'] = 5
ModActivity = ['NB','BC','O','S','A','I']

ModINPUTS = dfMod[col_inputs].values
ModOUTPUT = dfMod[['Activity']].values.ravel()

In [50]:
from sklearn import tree
from sklearn.metrics import zero_one_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Define tree parameters
# criterion='gini'
# splitter='best'
# max_depth=None
# min_samples_split=2
# min_samples_leaf=1
# min_weight_fraction_leaf=0.0
# max_features=None
# random_state=None
# max_leaf_nodes=None
# min_impurity_split=1e-07
# class_weight=None
# presort=False
X_train, X_test, Y_train, Y_test = train_test_split(ModINPUTS, ModOUTPUT, test_size=0.25, stratify=OUTPUT)
# X_train, X_test = INPUTS[train_index,:], INPUTS[test_index,:]
# Y_train, Y_test = OUTPUT[train_index], OUTPUT[test_index]
# Train Classifier
clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best', min_samples_split=2,
                                      min_samples_leaf=5, max_depth = 25)
clf.fit(X_train, Y_train)
# Test Classifier
Y_pred = clf.predict(X_test)
# Measure Classification Error
# zero_one_loss returns fractions of misclasssifications (0 is perfect)
f1 = f1_score(Y_test,Y_pred,average=None)
avgf1 = f1_score(Y_test,Y_pred,average='micro')    
error = zero_one_loss(Y_pred,Y_test)
print("Global F1: {0:.5f}, zero one loss: {1:.5f}".format(avgf1, error))
print("Depth: {0}, Node Count: {1}".format(clf.tree_.max_depth,clf.tree_.node_count))
print('\n'.join('{0}: {1:.5f}'.format(ModActivity[idx],k) for idx,k in enumerate(f1)))

Global F1: 0.94894, zero one loss: 0.05106
Depth: 25, Node Count: 7839
A: 0.93374
I: 0.95847


In [31]:
# PLOT TREE
import pydotplus
from IPython.display import Image

dot_data = tree.export_graphviz(clf, out_file=None, feature_names=col_inputs,class_names=Activity,filled=True, rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_svg("tree_MaxDepth10.svg")

True

In [14]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

## Random Forest Implementation

### Exhaustive Grid Search to Find Optimal parameters

In [3]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RFC

print(__doc__)

# Split the dataset into two equal parts
X_train, X_test, y_train, y_test = train_test_split(INPUTS, OUTPUT, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = { 
    'n_estimators': [5, 10, 15, 20, 30, 50, 100, 200],
    'max_depth': [15],
    'max_features': ['auto', 'sqrt', 'log2']
}

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(RFC(), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'max_depth': 15, 'max_features': 'auto', 'n_estimators': 100}

Grid scores on development set:

0.739 (+/-0.007) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 5}
0.746 (+/-0.014) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 10}
0.752 (+/-0.009) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 15}
0.750 (+/-0.007) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 20}
0.754 (+/-0.014) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 30}
0.756 (+/-0.007) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 50}
0.758 (+/-0.005) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 100}
0.756 (+/-0.005) for {'max_depth': 15, 'max_features': 'auto', 'n_estimators': 200}
0.738 (+/-0.013) for {'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 5}
0.751 (+/-

### Run Cross Validation with parameters found above that maximize F1 metric

In [3]:
# A random forest is a meta estimator that fits a number of decision tree 
# classifiers on various sub-samples of the dataset and use averaging to improve
# the predictive accuracy and control over-fitting. The sub-sample size is always
# the same as the original input sample size but the samples are drawn with 
# replacement if bootstrap=True (default).
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import zero_one_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from timeit import default_timer as timer
# Define parameters of classifier
nTrees = 100
maxDepth = 15

# Stratified K-Folds cross validation iterator
# Provides train/test indices to split data in train test sets.
# This cross-validation object is a variation of KFold that returns stratified folds. 
# The folds are made by preserving the percentage of samples for each class.
# Define variables
nfolds = 10 #Number of folds to use
averageF1 = 0.0 #running avg f1
averageError = 0.0 #running avg error

# Define train/test indices
skf = StratifiedKFold(n_splits=nfolds)
skf.shuffle = True
# Initialize lists
avgf1 = []
f1 = []
error = []
fi = [] # feature importance
rf = [] # random forest list
# Perform cross-validation
start = timer()
idx = 0;
for train_index, test_index in skf.split(INPUTS,OUTPUT):
    X_train, X_test = INPUTS[train_index,:], INPUTS[test_index,:]
    Y_train, Y_test = OUTPUT[train_index], OUTPUT[test_index]
    # Train Classifier
    rf.append(RandomForestClassifier(n_estimators=nTrees,max_depth=maxDepth))
    rf[idx].fit(X_train,Y_train)
    # Test Classifier
    Y_pred = rf[idx].predict(X_test)
    # Measure Classification Error
    # zero_one_loss returns fractions of misclasssifications (0 is perfect)
    error.append(zero_one_loss(Y_test,Y_pred))
    f1.append(f1_score(Y_test,Y_pred,average=None))
    avgf1.append(f1_score(Y_test,Y_pred,average='micro'))
    fi.append(rf[idx].feature_importances_)
    print("[Fold {0}] Global F1: {1:.5f}, zero one loss: {2:.5f}".format(idx+1, avgf1[idx], error[idx]))
    # print('\n'.join('{0}: {1:.5f}'.format(Activity[i],k) for i,k in enumerate(f1[idx])))
    # print('\n'.join('{0}: {1:.5f}'.format(col_inputs[i],k) for i,k in enumerate(fi[idx])))
    idx +=1
end = timer()
print("------------------------Computational time: %4.2f%s -------------------------------" % (end-start,' seconds'))
print("Average zero one loss: {0:.5f}s +- {1:0.5f}".format(np.mean(error),np.std(error)))
print("Average Global. F1: {0:.5f}s +- {1:0.5f}".format(np.mean(avgf1),np.std(avgf1)))
print("Average F1:")
print('\n'.join('{0}: {1:.5f} +- {2:.5f}'.format(Activity[if1],kf1[0],kf1[1]) for if1,kf1 in enumerate(zip(np.vstack(f1).mean(axis=0),np.vstack(f1).std(axis=0)))))
print("------------------------FEATURE IMPORTANCE-----------------------------------------")
print('\n'.join('{0}: {1:.5f} +- {2:.5f}'.format(col_inputs[ifi],kfi[0],kfi[1]) for ifi,kfi in enumerate(zip(np.vstack(fi).mean(axis=0),np.vstack(fi).std(axis=0)))))
# print out graphviz tree visualization
# tree0 = rf.estimators_[0];
# tree.export_graphviz(tree0, out_file='tree.dot') 
# dot -Tps tree.dot -o outfile.ps

[Fold 1] Global F1: 0.86617, zero one loss: 0.13383
[Fold 2] Global F1: 0.86504, zero one loss: 0.13496
[Fold 3] Global F1: 0.86013, zero one loss: 0.13987
[Fold 4] Global F1: 0.86147, zero one loss: 0.13853
[Fold 5] Global F1: 0.86692, zero one loss: 0.13308
[Fold 6] Global F1: 0.86534, zero one loss: 0.13466
[Fold 7] Global F1: 0.85028, zero one loss: 0.14972
[Fold 8] Global F1: 0.87088, zero one loss: 0.12912
[Fold 9] Global F1: 0.85389, zero one loss: 0.14611
[Fold 10] Global F1: 0.86277, zero one loss: 0.13723
------------------------Computational time: 45.06 seconds -------------------------------
Average zero one loss: 0.13771s +- 0.00590
Average Global. F1: 0.86229s +- 0.00590
Average F1:
b: 0.83205 +- 0.01553
bc: 0.80552 +- 0.01458
f: 0.81302 +- 0.01958
fd: 0.64787 +- 0.02396
go: 0.69037 +- 0.02735
i: 0.93779 +- 0.00496
og: 0.78665 +- 0.01803
sg: 0.56587 +- 0.02172
t: 0.60014 +- 0.02336
wi: 0.68565 +- 0.01370
wo: 0.83534 +- 0.01676
------------------------FEATURE IMPORTANCE---

### Plot importance of features 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 

N = len(col_inputs) # number of features
ind = np.arange(N)  # the x locations for the groups
width = 0.05        # the width of the bars
bars = []
# Plot the feature importances of the forest
fig, ax = plt.subplots()
c=iter(cm.rainbow(np.linspace(0,1,len(rf))))
for i in range(0,len(rf)):
    importances = fi[i]
    std = np.std([tree.feature_importances_ for tree in rf[i].estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    bars.append(ax.bar(ind + width*i, importances[indices], width, color=next(c), yerr=std[indices]))

    
# add some text for labels, title and axes ticks
ax.set_ylabel('Importance')
ax.set_title('Feature Importance')
ax.set_xticks(ind + width*5)
ax.set_xticklabels(col_inputs[i] for i in indices)
ax.set_ylim(0,1)
ax.legend((b[0] for b in bars), ("Fold {0}".format(idx+1) for idx,k in enumerate(bars)))

plt.show()

### Try One vs Rest Approach to find important features per task

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Define parameters of classifier
nTrees = 20
maxDepth = 15

X_train, X_test, Y_train, Y_test = train_test_split(INPUTS, OUTPUT, test_size=0.25, stratify=OUTPUT)
clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=nTrees,max_depth=maxDepth))
clf.fit(INPUTS,OUTPUT)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 

N = len(col_inputs) # number of features
ind = np.arange(N)  # the x locations for the groups
width = 0.05        # the width of the bars
bars = []
# Plot the feature importances of the forest
fig, ax = plt.subplots()
c=iter(cm.rainbow(np.linspace(0,1,len(Activity))))
for i in range(0,len(Activity)):
    rf = clf.estimators_[i]
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    bars.append(ax.bar(ind + width*i, importances[indices], width, color=next(c), yerr=std[indices]))

    
# add some text for labels, title and axes ticks
ax.set_ylabel('Importance')
ax.set_title('Feature Importance per Task (OnevsRest)')
ax.set_xticks(ind + width*5)
ax.set_xticklabels(col_inputs[i] for i in indices)
ax.set_ylim(0,1)
ax.legend((b[0] for b in bars), Activity)

plt.show()

In [ ]:
clf.classes_

## Decision Tree

In [ ]:
from sklearn import tree
from sklearn.metrics import zero_one_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import f1_score
from timeit import default_timer as timer

# Define tree parameters
# criterion='gini'
# splitter='best'
# max_depth=None
# min_samples_split=2
# min_samples_leaf=1
# min_weight_fraction_leaf=0.0
# max_features=None
# random_state=None
# max_leaf_nodes=None
# min_impurity_split=1e-07
# class_weight=None
# presort=False

# Define train/test indices
skf = StratifiedKFold(OUTPUT,n_folds=3)
skf.shuffle = 'True'
# Initialize lists
avgf1 = []
f1 = []
error = []
idx = 0
averageError = 0.0

start = timer()
for train_index, test_index in skf:
    X_train, X_test = INPUTS[train_index,:], INPUTS[test_index,:]
    Y_train, Y_test = OUTPUT[train_index], OUTPUT[test_index]
    # Train Classifier
    clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=5, min_samples_split=2,
                                      min_samples_leaf=5)
    clf.fit(X_train, Y_train)
    # Test Classifier
    Y_pred = clf.predict(X_test)
    # Measure Classification Error
    # zero_one_loss returns fractions of misclasssifications (0 is perfect)
    f1.append(f1_score(Y_test,Y_pred,average=None))
    avgf1.append(f1_score(Y_test,Y_pred,average='micro'))    
    error.append(zero_one_loss(Y_pred,Y_test))
    print("[Fold {0}] Global F1: {1:.5f}, zero one loss: {2:.5f}".format(idx, avgf1[idx], error[idx]))
    print("Depth: {0}, Node Count: {1}".format(clf.tree_.max_depth,clf.tree_.node_count))
    print('\n'.join('{0}: {1:.5f}'.format(*k) for k in enumerate(f1[idx])))
    idx +=1
end = timer()
print("---------------------------------Computational time: %4.2f%s -------------------------------" % (end-start,' seconds'))
print("Average error: %.5f%s" % (np.mean(error),'%'))
print("Average Global F1: %.5f%s" % (np.mean(avgf1),'%'))
print("Average F1:")
print('\n'.join('{0}: {1:.5f}'.format(*k) for k in enumerate(np.vstack(f1).mean(axis=0))))

In [ ]:
# For INPUTS = []'HeadToThorax_mm','Corpulence_mm','Dist'] :
# If max_depth = None, then avg. tree depth is 48 with ~45000 nodes (pretty bad overfitting)
# If max_depth = 5, avg # of nodes is 55 (pretty bad overfitting)

# More importantly, there are very few instances of certain labels (activities):
a = ['b', 'bc', 'f', 'fd', 'go', 'i', 'og', 'sg', 't', 'wi', 'wo']
num_rows =  len(df.Activity)

for activity in df.Activity.unique():
    temp = df.loc[df['Activity'] == activity]
    print('Activity {0}: {1} rows = {2:.2f}%'.format(a[int(activity)],len(temp),100*(float(len(temp))/num_rows)))

In [ ]:
 len(df.Activity)
